In [76]:
import sys
from operator import itemgetter

In [77]:
# Read data

path = "testcase/wc_data.txt"

In [78]:
# Sorting Algorithm used to sort the output of the reducer by the key

def hash_function(key):

    if key[0].lower() < 'm':
        return 0
    else:
        return 1

In [79]:
# mapper.py

mapper_output = []

with open(path, 'r') as file:
    for line in file:
        line = line.strip()
        words = line.split()
        
        for word in words:

            partition = hash_function(word)
            mapper_output.append((word, 1, partition))

print("Mapper Output (with partition):")
for output in mapper_output:
    print(f'{output[0]}\t{output[1]}\tPartition: {output[2]}')

Mapper Output (with partition):
I	1	Partition: 0
love	1	Partition: 0
drink	1	Partition: 0
craft	1	Partition: 0
beer	1	Partition: 0
and	1	Partition: 0
larger	1	Partition: 0
beer	1	Partition: 0
in	1	Partition: 0
Taiwan	1	Partition: 1


In [80]:
# Shuffle and Sort

partitioned_mapper_output = {}

for word, count, partition in mapper_output:
    if partition not in partitioned_mapper_output:
        partitioned_mapper_output[partition] = []
    partitioned_mapper_output[partition].append((word, count))

In [81]:
# Sort by word (key) across partitions
sorted_output = []
for partition in partitioned_mapper_output:
    partitioned_mapper_output[partition].sort(key=itemgetter(0))
    sorted_output.extend(partitioned_mapper_output[partition])

# Sort the final combined output by the word
sorted_output.sort(key=itemgetter(0))

In [82]:
# Reducer

print("\nReducer Output:")
current_word = None
current_count = 0

for word, count in sorted_output:
    if current_word == word:
        current_count += count
    else:
        if current_word:
            print(f'{current_word}\t{current_count}')
        current_count = count
        current_word = word

# Print the last word count
if current_word == word:
    print(f'{current_word}\t{current_count}')


Reducer Output:
I	1
Taiwan	1
and	1
beer	2
craft	1
drink	1
in	1
larger	1
love	1
